In [1]:
!pip install unsloth --upgrade

In [2]:
!pip install datasets

In [5]:
#datasetimizi yukleyelim
from datasets import load_dataset

dataset = load_dataset("Amod/mental_health_counseling_conversations")
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import torch
from trl import SFTTrainer #Hugging Face'in SFTTrainer sınıfı, ince ayar yapmak için bir araç sağlar.
from transformers import TrainingArguments, TextStreamer #Hugging Face Transformers kitaplığından eğitim argümanları ve metin akışını yöneten sınıflar dahil ediliyor.
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported #bfloat16 desteğini kontrol ediyor.
# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

from datasets import load_dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

data=[]
for x in dataset['train']:
  data.append(x)

df=pd.DataFrame(data)

df['Context_length'] = df['Context'].apply(len)
filtered_data = df[df['Context_length'] <= 1500]

filtered_data['ln_Response']=filtered_data['Response'].apply(len)
filtered_data = filtered_data[filtered_data['ln_Response'] <= 4000]

max_seq_length=5020
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=False,  # Load in full precision (FP32)
    dtype=torch.torch.bfloat16,  # torch.bfloat16
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

data_prompt = """Analyze the provided text from a mental health perspective. Identify any indicators of emotional distress, coping mechanisms, or psychological well-being. Highlight any potential concerns or positive aspects related to mental health, and provide a brief explanation for each observation.

### Input:
{}

### Response:
{}"""



==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


trainable params: 11,272,192 || all params: 1,247,086,592 || trainable%: 0.9039
None


In [ ]:
filtered_data = filtered_data.drop(columns=['Context_length', 'ln_Response'])


In [8]:
filtered_data

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [10]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["Context"]
    outputs      = examples["Response"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Veriyi formatlandırın ve dataseti hazırlayın.
training_data = Dataset.from_pandas(filtered_data).map(formatting_prompt, batched=True)

# SFT Trainer tanımlayın.
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        num_train_epochs=50,
        fp16=False, # SFT Trainer fp16 desteği desteklenir
        bf16=True,  # SFT Trainer bfloat16 desteği desteklenir
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

# Eğitimi başlatın.
trainer.train()

Map:   0%|          | 0/3480 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 227 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 150
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,8.837400
2,8.718900
3,8.671200
4,6.511400
5,8.640700
6,8.562100
7,8.451300
8,6.329700
9,8.306700
10,8.270400


TrainOutput(global_step=150, training_loss=4.519476612408956, metrics={'train_runtime': 2946.6875, 'train_samples_per_second': 3.852, 'train_steps_per_second': 0.051, 'total_flos': 2.5283134253162496e+17, 'train_loss': 4.519476612408956, 'epoch': 37.53333333333333})

In [11]:
model.save_pretrained("model/1B_finetuned_llama3.2")
tokenizer.save_pretrained("model/1B_finetuned_llama3.2")

('model/1B_finetuned_llama3.2/tokenizer_config.json',
 'model/1B_finetuned_llama3.2/special_tokens_map.json',
 'model/1B_finetuned_llama3.2/tokenizer.json')

In [12]:
text="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"



In [13]:
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]
print("Answer of the question is:", answer)


Answer of the question is: 
I'm sorry you're going through this.  It's a common feeling, but it's also a self-destructive thought.  It's a self-protection mechanism that the brain is using to keep you safe in the short term.  It's not a permanent change.  You can change that feeling of worthless.  I'm going to give you two tips.  The first tip is to write down all the things you've done to try to fix your issues.  You'll be surprised how much energy you have when you realize you're doing something to fix your issues.  The second tip is to ask yourself, "What is making me feel worthless?"  It's not the issue itself, but it's the thought that's making you feel that way.  Once you find that, ask yourself "What would happen if I didn't feel worthless?"  You might end up doing something you never done before.  You might find a new hobby or a new job.  You might meet someone new that will like you more for being at peace with yourself.  The more you believe you can fix it, the more you will 